In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

class DataFrameAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("DataFrame Analyzer")

        # Frame pour les widgets
        self.frame = tk.Frame(root)
        self.frame.pack(padx=10, pady=10)

        # Bouton pour charger un fichier
        self.load_button = tk.Button(self.frame, text="Charger un fichier", command=self.load_file)
        self.load_button.grid(row=0, column=0, pady=(0, 10))

        # Liste des opérations disponibles
        self.operations = ["Valeur la plus fréquente", "Valeurs distinctes"]

        # Label et Menu déroulant pour chaque colonne
        self.column_labels = []
        self.column_menus = []
        self.column_results = []
        self.columns = []

        # Bouton pour exécuter les opérations
        self.analyze_button = tk.Button(self.frame, text="Analyser", command=self.analyze)
        self.analyze_button.grid(row=0, column=2, pady=(0, 10))

        # Bouton pour enregistrer les résultats
        self.save_button = tk.Button(self.frame, text="Enregistrer les résultats", command=self.save_results)
        self.save_button.grid(row=0, column=3, pady=(0, 10))

        # DataFrame pour stocker les résultats
        self.results_df = pd.DataFrame()

    def load_file(self):
        # Ouvrir une boîte de dialogue pour sélectionner un fichier
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv"), ("Excel Files", "*.xlsx")])

        if file_path:
            # Charger le fichier dans un DataFrame
            if file_path.endswith('.csv'):
                self.df = pd.read_csv(file_path)
            elif file_path.endswith('.xlsx'):
                self.df = pd.read_excel(file_path)
            else:
                tk.messagebox.showerror("Erreur", "Format de fichier non pris en charge.")
                return

            # Créer des widgets pour chaque colonne
            self.create_column_widgets()

    def create_column_widgets(self):
        # Supprimer les anciens widgets s'ils existent
        for label, menu, result in zip(self.column_labels, self.column_menus, self.column_results):
            label.grid_forget()
            menu.grid_forget()
            result.grid_forget()

        self.column_labels = []
        self.column_menus = []
        self.column_results = []
        self.columns = []

        # Créer des widgets pour chaque colonne du DataFrame
        for i, column in enumerate(self.df.columns):
            label = tk.Label(self.frame, text=f"Colonne {column}:")
            label.grid(row=i + 1, column=0, sticky='w')

            menu = tk.OptionMenu(self.frame, tk.StringVar(), *self.operations)
            menu.grid(row=i + 1, column=1)

            result = tk.Label(self.frame, text="")
            result.grid(row=i + 1, column=2)

            self.column_labels.append(label)
            self.column_menus.append(menu)
            self.column_results.append(result)
            self.columns.append(column)

    def analyze(self):
        if hasattr(self, 'df'):
            results_data = []

            for column, menu, result in zip(self.columns, self.column_menus, self.column_results):
                operation = menu.cget("text")
                if operation == "Valeur la plus fréquente":
                    result_text = f"Valeur la plus fréquente: {self.df[column].mode().iloc[0]}"
                    results_data.append([column, result_text])
                elif operation == "Valeurs distinctes":
                    result_text = f"Valeurs distinctes: {', '.join(map(str, self.df[column].unique()))}"
                    results_data.append([column, result_text])
                else:
                    result_text = "Opération non prise en charge"

                result.config(text=result_text)

            # Mettre à jour le DataFrame des résultats
            self.results_df = pd.DataFrame(results_data, columns=['Colonne', 'Résultat'])

    def save_results(self):
        if not self.results_df.empty:
            # Ouvrir une boîte de dialogue pour sélectionner l'emplacement de sauvegarde
            save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])

            if save_path:
                # Enregistrer le DataFrame des résultats dans un fichier CSV
                self.results_df.to_csv(save_path, index=False)
                tk.messagebox.showinfo("Enregistrement réussi", "Les résultats ont été enregistrés avec succès.")
        else:
            tk.messagebox.showinfo("Aucun résultat", "Aucun résultat à enregistrer.")

if __name__ == "__main__":
    root = tk.Tk()
    app = DataFrameAnalyzerApp(root)
    root.mainloop()
